In [1]:
from pathlib import Path
import os
import sys
import gc
import shutil
import json
import math
import uuid
from collections import defaultdict
import numpy as np
import pandas as pd
from spacy.lang.en import English
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, NamedTuple, Callable, Iterable, Set, Optional, Any
import scml
from mylib.preprocess.en import BasicPreprocessor
from mylib import datageneration as dg

In [2]:
tim = scml.Timer()
tim.start()
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
    for i in range(torch.cuda.device_count()):
        print(f"device={i}, {torch.cuda.get_device_name(i)}")
        print('Mem Allocated:', round(torch.cuda.memory_allocated(i)/1024**3,1), 'GB')
        print('Mem Cached:   ', round(torch.cuda.memory_reserved(i)/1024**3,1), 'GB')
else:
    print("cpu")

device=0, NVIDIA GeForce RTX 4070 Ti SUPER
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB
device=1, NVIDIA GeForce RTX 4070 Ti SUPER
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB


In [3]:
min_entities = 2
tokenizer = English().tokenizer
filepaths = [
    Path("input/part01.parquet"),
    Path("input/part02.parquet"),
    Path("input/part03.parquet"),
    Path("input/part04.parquet"),
    Path("input/part05.parquet"),
    Path("input/part06.parquet"),
    Path("input/part07.parquet"),
    Path("input/part08.parquet"),
    Path("input/part09.parquet"),
    Path("input/part10.parquet"),
]

In [4]:
df = None
for fp in filepaths:
    if df is None:
        df = pd.read_parquet(fp)
        continue
    tmp = pd.read_parquet(fp)
    df = pd.concat([df, tmp], axis=0, ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2530 entries, 0 to 2529
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   text            2530 non-null   object
 1   prompt          2530 non-null   object
 2   street_address  2530 non-null   object
 3   student_name    2530 non-null   object
 4   username        2530 non-null   object
 5   personal_url    2530 non-null   object
 6   model           2530 non-null   object
dtypes: object(7)
memory usage: 138.5+ KB


In [5]:
k=10
df.sample(k).head(k)

,text,prompt,street_address,student_name,username,personal_url,model
1727,"Title: The Importance of Reflection in Crisis Response and Humanitarian Aid: An Examination through Real-Life Examples\n\nIn the field of crisis response and humanitarian aid, reflection is an essential tool in addressing complex challenges and preventing misrepresentation in the design thinking process. By continually examining and reexamining our actions, assumptions, and decisions, we can ensure that we are effectively meeting the needs of those we serve while maintaining the integrity of our work. This essay will explore the importance of reflection through three real-life examples: a crisis near Bedok North Road, the experience of an individual named ""the_nuruliamazizahofficial,"" and a case involving ""foursquare.com/officialnurulxxxazizah.""\n\nNear Bedok North Road, at a school frequented by Nurul Azizah, a significant crisis unfolded, highlighting the critical role of reflection in crisis response. As aid workers arrived on the scene, they quickly realized that their initial assessment of the situation was incomplete. They had assumed that food and water were the primary needs of the affected community; however, upon further reflection and engagement with those impacted, they discovered that there was also a pressing need for medical supplies and psychological support. This realization allowed them to adjust their approach, providing more comprehensive aid and avoiding the potential harm caused by misrepresentation of the community's needs.\n\nThe experience of ""the_nuruliamazizahofficial"" serves as another example of the power of reflection in crisis response and humanitarian aid. Following a series of public statements about her relief efforts, she faced criticism from various stakeholders. Through thoughtful reflection, she realized that some of her words and actions may have unintentionally perpetuated stereotypes and oversimplified the complex issues at hand. As a result, she actively sought feedback from others, modified her messaging, and adopted a more inclusive and nuanced approach in her communications. This demonstrates the capacity of reflection to prevent misrepresentation, promote learning, and ultimately improve the effectiveness of humanitarian interventions.\n\nLastly, the case of ""foursquare.com/officialnurulxxxazizah"" reveals the significance of ongoing self-assessment in ensuring the credibility and accuracy of information disseminated during a crisis. With allegations of misinformation surfacing regarding her organization's activities, this individual took the opportunity","[INST] Write a short essay on the following topic:\n In Crisis Response and Humanitarian Aid, how does reflection contribute in tackling challenges and preventing misrepresentation in the design thinking process?\n Intersperse the following information in different parts of the essay:\n 1. Give an example that occurred near Bedok North Road (Block 185), at a school frequented by Nurul Azizah\n 2. Give an example citing someone named the__nuruliamazizahofficial\n 3. Give an example citing: foursquare.com/officialnurulxxxazizah [/INST]\n",Bedok North Road (Block 185),Nurul Azizah,the__nuruliamazizahofficial,foursquare.com/officialnurulxxxazizah,1
1874,"Design thinking is a problem-solving approach that involves empathizing with the user, defining the problem, ideating solutions, creating prototypes, and testing those prototypes. This iterative process encourages creativity and collaboration among team members as they work towards developing innovative solutions to complex challenges.\n\nIn ocean conservation and coastal management, brainstorming and prototyping are particularly important stages in the design thinking process. These stages allow teams to generate a wide range of ideas and turn them into tangible solutions, fostering both creativity and teamwork.\n\nFor instance, consider the situation near Yishun Central (Block 302) where Amirah Syazwa

In [6]:
pre = BasicPreprocessor()
rows = []
for t in tqdm(df.itertuples()):
    street_address = str(getattr(t, "street_address"))
    student_name = str(getattr(t, "student_name"))
    username = str(getattr(t, "username"))
    personal_url = str(getattr(t, "personal_url"))
    text = pre(getattr(t, "text"))
    sp_tokens = tokenizer(text)
    tokens = [token.text for token in sp_tokens]
    trailing_whitespace = [bool(token.whitespace_) for token in sp_tokens]
    res = dg.labels(
        text=text, 
        tokens=sp_tokens, 
        street_address=street_address,
        student_name=student_name,
        username=username,
        personal_url=personal_url,
    )
    total = res.n_street_address + res.n_student_name + res.n_username + res.n_personal_url
    if total < min_entities:
        continue
    rows.append({
        "document": f"rh_{uuid.uuid4().hex}",
        "full_text": text,
        "tokens": tokens,
        "trailing_whitespace": trailing_whitespace,
        "labels": res.labels, 
        "n_street_address": res.n_street_address,
        "n_student_name": res.n_student_name,
        "n_username": res.n_username,
        "n_personal_url": res.n_personal_url,
        "street_address": street_address,
        "student_name": student_name,
        "username": username,
        "personal_url": personal_url,
    })


2530it [00:01, 1352.11it/s]


In [7]:
df = pd.DataFrame.from_records(rows)
cols = ["n_street_address", "n_student_name", "n_username", "n_personal_url"]
df[cols] = df[cols].astype(np.int8)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2242 entries, 0 to 2241
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   document             2242 non-null   object
 1   full_text            2242 non-null   object
 2   tokens               2242 non-null   object
 3   trailing_whitespace  2242 non-null   object
 4   labels               2242 non-null   object
 5   n_street_address     2242 non-null   int8  
 6   n_student_name       2242 non-null   int8  
 7   n_username           2242 non-null   int8  
 8   n_personal_url       2242 non-null   int8  
 9   street_address       2242 non-null   object
 10  student_name         2242 non-null   object
 11  username             2242 non-null   object
 12  personal_url         2242 non-null   object
dtypes: int8(4), object(9)
memory usage: 166.5+ KB


In [8]:
df.describe(percentiles=percentiles)

,n_street_address,n_student_name,n_username,n_personal_url
count,2242.000000,2242.000000,2242.000000,2242.000000
mean,1.637377,1.421053,0.648974,0.756467
std,0.708269,0.908222,0.901371,0.584277
min,0.000000,0.000000,0.000000,0.000000
1%,1.000000,0.000000,0.000000,0.000000
5%,1.000000,0.000000,0.000000,0.000000
10%,1.000000,1.000000,0.000000,0.000000
20%,1.000000,1.000000,0.000000,0.000000
30%,1.000000,1.000000,0.000000,0.000000
40%,1.000000,1.000000,0.000000,1.000000


In [9]:
for row in rows[:3]:
    print("===========")
    print(f"street_address={row['street_address']}")
    print(f"student_name={row['student_name']}")
    print(f"username={row['username']}")
    print(f"personal_url={row['personal_url']}")
    lbs = row["labels"]
    tokens = row["tokens"]
    ts = []
    for i in range(len(lbs)):
        if lbs[i]!="O":
            ts.append(tokens[i])
    print(ts)

street_address=Choa Chu Kang Avenue 4 (Block 444)
student_name=Rahman bin Hassan
username=_4807.rahman-iam--hassan
personal_url=periscope.tv/rahmanhassanxxx
['Choa', 'Chu', 'Kang', 'Avenue', '4', 'Rahman', 'bin', 'Hassan', '_', '4807.rahman', '-', 'iam', '--', 'hassan', '_', '4807.rahman', '-', 'iam', '--', 'hassan', 'Periscope.tv/rahmanhassanxxx', 'Rahman', 'bin', 'Hassan', '_', '4807.rahman', '-', 'iam', '--', 'hassan']
street_address=Jurong West Street 92 (Block 922)
student_name=Nicholas Sami
username=__iamNicholas_xxx.Sami
personal_url=blogspot.me/@iamnicholassami
['Jurong', 'West', 'Street', '92', '(', 'Block', '922', ')', '_', '_', 'iamNicholas_xxx', '.', 'Sami', 'Blogspot.me/@iamnicholassami']
street_address=Jalan Eunos (Eunos Mansion)
student_name=Nina Setiawan
username=the-nina3719setiawan
personal_url=sinaweibo.com/ninaiamsetiawanthe
['Jalan', 'Eunos', 'Nina', 'Setiawan', 'the', '-', 'nina3719setiawan', 'The', '-', 'nina3719setiawan', 'the', '-', 'nina3719setiawan', 'sinawei

In [10]:
%%time
with open("output/rh02.json", "w") as f:
    json.dump(rows, f)
df.to_parquet(f"output/rh02.parquet", index=False)
assert df.notna().all(axis=None)

CPU times: user 571 ms, sys: 42.3 ms, total: 613 ms
Wall time: 903 ms


In [11]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 0:00:03.172229
